In [ ]:
!pip install /kaggle/input/segmentation-whl/timm-0.6.12-py3-none-any.whl
!pip install /kaggle/input/segmentation-whl/efficientnet_pytorch-0.7.1-py3-none-any.whl
!pip install /kaggle/input/segmentation-whl/pretrainedmodels-0.7.4-py3-none-any.whl
!pip install /kaggle/input/segmentation-whl/segmentation_models_pytorch-0.3.2-py3-none-any.whl
# !pip install -q -U segmentation-models-pytorch albumentations > /dev/null
import segmentation_models_pytorch as smp

In [ ]:
import torch
import torch.nn as nn
import albumentations as A
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import pathlib
import cv2

from pathlib import Path
from albumentations import *

from tqdm.notebook import tqdm

In [ ]:
test_data_path = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/test'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def get_training_augmentation():
    train_transform = A.Compose([
        A.Resize(384, 384, interpolation=cv2.INTER_CUBIC),
#         A.HueSaturationValue(p=0.5),
#         A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), p=0.5),
#         A.ChannelShuffle(p=1),
        A.OneOf(
            [
                A.HorizontalFlip(p=1),
                A.VerticalFlip(p=1),
                A.RandomRotate90(p=1),
            ],p=0.5,),
        ])
    
    return train_transform

def get_val_augmentation():   
    test_transform = [
        A.PadIfNeeded(min_height=256, min_width=256, always_apply=True, border_mode=0),
        A.Resize(384, 384, interpolation=cv2.INTER_CUBIC),
    ]
    return A.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1)

def get_preprocessing(preprocessing_fn=None):
    _transform = []
    if preprocessing_fn:
        _transform.append(A.Lambda(image=preprocessing_fn))
    _transform.append(A.Lambda(image=to_tensor, mask=to_tensor))
        
    return A.Compose(_transform)

In [ ]:
import segmentation_models_pytorch.utils.metrics
import segmentation_models_pytorch.utils

# ENCODER = 'resnet50'
ENCODER = 'efficientnet-b1'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['Contrail']
ACTIVATION = 'sigmoid'

model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=None,
    classes = len(CLASSES),
    activation = ACTIVATION,
)

# preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

TRAINING = True
Epochs = 30
loss = smp.utils.losses.DiceLoss()
metrics = [smp.utils.metrics.IoU(threshold=0.5)]
optim = torch.optim.Adam([dict(params=model.parameters(), lr=0.0001)])
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim, T_0=1, T_mult=2, eta_min=5e-5)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, max_lr=0.001, epochs=Epochs, steps_per_epoch=1284)

In [ ]:
import torchvision.transforms as T

def false_color(band11, band14, band15):
    def normalize(band, bounds):
        return (band - bounds[0]) / (bounds[1] - bounds[0])    
    _T11_BOUNDS = (243, 303)
    _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
    _TDIFF_BOUNDS = (-4, 2)
    r = normalize(band15 - band14, _TDIFF_BOUNDS)
    g = normalize(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize(band14, _T11_BOUNDS)
    return np.clip(np.stack([r, g, b], axis=2), 0, 1)

class ContrailTestDataset(Dataset):
    def __init__(self, images_dir, ids, augmentation=None, preprocessing=None):
        super().__init__()
        self.image_path = images_dir
        self.ids = ids
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
#         self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        
    def __len__(self):
        return len(self.ids)
        
    def __getitem__(self, idx):
        image_path  = f"{self.image_path}/{self.ids[idx]}"
        band11 = np.load(f"{image_path}/band_11.npy")[..., 4]
        band14 = np.load(f"{image_path}/band_14.npy")[..., 4]
        band15 = np.load(f"{image_path}/band_15.npy")[..., 4]
        image = false_color(band11, band14, band15)
        
        if self.augmentation:
            sample = self.augmentation(image=image)
            image = sample['image']
            
        if self.preprocessing:
            image = np.transpose(image, (2, 0, 1))
            
        image = image.astype(np.float32)
        image = torch.as_tensor(image)
#         image = self.normalize_image(image)
        
#         image = (image - image.min()) / (image.max() - image.min())
        # image /= 255.0
            
        return image

In [ ]:
model = torch.load('/kaggle/input/contrail-model-save2/Resize_384_Epoch130.pth', map_location=DEVICE)

In [ ]:
ids = os.listdir(test_data_path)
test_ds = ContrailTestDataset(test_data_path, ids, get_val_augmentation(), preprocessing=True)
test_dl = DataLoader(test_ds, batch_size=1, num_workers=4, shuffle=False, pin_memory=True)

In [ ]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s


def rle_decode(mask_rle, shape=(256, 256)):
    '''
    mask_rle: run-length as string formatted (start length)
              empty predictions need to be encoded with '-'
    shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''

    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    if mask_rle != '-': 
        s = mask_rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
    return img.reshape(shape, order='F')  # Needed to align to RLE direction

In [ ]:
def submit_to_csv(test_ids, y_pred):
    
    y_encoded = [list_to_string(rle_encode(y)) for y in y_pred]
    
    all_test_paths = list(Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming/test').glob("*"))
    
    sub_df = pd.DataFrame({'record_id': [path.stem for path in all_test_paths], 'encoded_pixels': y_encoded})
    
    print("Submitted")
    return sub_df

In [ ]:
out_rle = []
test_ids = []
preds=[]
model.eval()


with torch.inference_mode():
    for idx, image in enumerate(tqdm(test_dl)):
        print("Test idx:", idx)
        image = image.to(DEVICE, dtype=torch.float)
        
        pred = model(image)
        
        pred = torch.nn.functional.interpolate(pred, size=256, mode='bilinear')
#         print(pred.shape)
        
        pred = (nn.Sigmoid()(pred)>0.5).double()
    
        out_reversed = np.array([o.cpu().permute(1,2,0).numpy() for o in pred])
        for e in out_reversed:
            out_rle.append(e)

In [ ]:
test_ids = list(os.listdir("/kaggle/input/google-research-identify-contrails-reduce-global-warming/test"))

In [ ]:
sub_df= submit_to_csv(test_ids, out_rle)

In [ ]:
sub_df

In [ ]:
sub_df.to_csv("submission.csv",index=False)